In [ ]:
#Import required packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlsauce as ms
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from time import time
from os import chdir
from sklearn import metrics


In [ ]:
DATASET="Retina_0_0"
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET
outpath = basepath + "\\Outputs"
original_df=pd.read_csv(outpath+"\cell_info1.csv")  
original_df=original_df.rename(columns={"Unnamed: 0":"my_Index"})

In [ ]:
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET
outpath = basepath + "\\CellProfiler_Output"
pre_df=pd.read_csv(outpath+"\\BF_cells_on_grid_pre.csv") 

df=pd.read_csv(outpath+"\BF_cells_on_grid.csv")  
File_raw=df["FileName_Marker_image"].to_numpy()
Image_number=df["ImageNumber"].to_numpy()
File=np.array([int(i.split("Retina_0_0_Ch1")[1][:-4]) for i in File_raw])


In [ ]:
#This nightmarish code makes sense of the output from cell profiler. This took more than 24h to figure out. 

large_array=[]
for file in np.unique(File):
    if file!=11:  #File 11 has two cells with the same index, we ignore file 11 thus removing 1600 cells.
        file_locs=np.where(File==file)
        s_df=df.iloc[file_locs[0]]
        i_num=int(np.unique(Image_number[file_locs[0]]))
        s_pre_df=pre_df[pre_df["ImageNumber"]==i_num]
        X=s_pre_df["Location_Center_X"].to_numpy()//64
        Y=s_pre_df["Location_Center_Y"].to_numpy()//64
        Actual_Index=X*16+Y
        assert(len(Actual_Index)==len(np.unique(Actual_Index)))
        Actual_Index=Actual_Index.astype(int)
        Old_Index=s_pre_df["Number_Object_Number"].to_numpy()
        new_df=s_df.iloc[:,8:]
        features=new_df.to_numpy()
        for n,column in enumerate(new_df):
           # print("File:",file,"Column",n,"/",len(new_df.columns))
            a=features[:,n]
            Actual_Index=np.sort(Actual_Index)
            I=np.arange(len(a))
            junk=np.where(np.isnan(features[:,0])==1)[0]
            II=[]
            for i in I:
                count=0
                if i not in Actual_Index:
                  i=junk[count]
                  count+=1
                  II.append(np.nan)
                else:
                  II.append(np.float(a[np.where(Actual_Index==i)[0]]))
            II=np.array(II)
            II=II.reshape(100,16)
            II=II.T.flatten()
            new_df[column]=II
        latest=new_df.to_numpy()
        if len(large_array)==0:
            large_array=latest
        else:
            large_array=np.vstack((large_array,latest))

X=large_array
Y=original_df["Scaled_Ch07"].to_numpy()
Y=np.hstack((Y[:11*1600],Y[12*1600:]))  #Skip values corresponding to 11th file
ID=original_df["Cell_ID"].to_numpy()
ID=np.hstack((ID[:11*1600],ID[12*1600:]))

assert len(Y)==len(X)

idx_to_keep=np.sum(np.isnan(X),axis=1)==0
X=X[idx_to_keep]
Y=Y[idx_to_keep]
ID=ID[idx_to_keep]

final_df=pd.DataFrame()
final_df["Scaled_Ch07"]=Y
final_df["Cell_ID"]=ID
for n,column in enumerate(new_df):
    final_df[column]=X[:,n]

#final_df.to_csv(basepath+"\\Outputs"+"\\Cell_features.csv",index = False, header=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr

DATASET="Retina_0_0"
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET

my_df=pd.read_csv(basepath+"\\Outputs"+"\\Cell_features.csv")
Y=my_df["Scaled_Ch07"]
ID=my_df["Cell_ID"]
X=my_df.iloc[:,2:].to_numpy()

scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)


# rank=np.linalg.matrix_rank(X)
# from sklearn.decomposition import PCA
# pca = PCA(n_components=rank)
# X=pca.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)
from sklearn.linear_model import LinearRegression, Ridge

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred=reg.predict(X_test)
print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))
plt.scatter(y_test,y_pred,s=1)
print(spearmanr(y_pred,y_test)[0])

In [ ]:
names=my_df.columns[2:]
indexes=["Marker" not in name for name in names]
X=X[:,indexes]
names=names[indexes]

In [ ]:
import teller as tr
plt.figure(figsize=(10,100))
explainer = tr.Explainer(obj=reg)
explainer.fit(X_test, y_test[:,None], X_names=names, method="avg")
explainer.plot(what="average_effects")
plt.savefig(basepath+"\\Report"+"\\CellProfilerfeatures.png",bbox_inches="tight")

In [ ]:
###Random Forest

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


print(np.shape(X))
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2)


reg=RandomForestRegressor(n_estimators=100)
reg.fit(X_train, y_train)
y_pred=reg.predict(X_test)
y_train_pred=reg.predict(X_train)
print(spearmanr(y_train_pred,y_train)[0])
print(spearmanr(y_pred,y_test)[0])

#plt.scatter(y_test,y_pred,s=1)

In [ ]:
#LSBoost


import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston, load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from time import time
from os import chdir
from sklearn import metrics
import mlsauce as ms


print(X.shape,Y.shape)

# split data into training test and test set
np.random.seed(15029)
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.2)

obj = ms.LSBoostRegressor(tolerance=1e-2, activation="relu6", col_sample=1, row_sample=1,n_estimators=50,n_hidden_features=50,dropout=0.5,reg_lambda=0,backend="gpu",learning_rate=0.5)
#obj = ms.RidgeRegressor()
print(obj.get_params())
start = time()
obj.fit(X_train, y_train)

#print(np.sqrt(np.mean(np.square(obj.predict(X_test) - y_test))))

print("test",spearmanr(obj.predict(X_test),y_test)[0])
print("train",spearmanr(y_train,obj.predict(X_train))[0])
